<a href="https://colab.research.google.com/github/LombardoAndrea195/Beam_examples/blob/main/Beam_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### A Beam pipeline is a graph (specifically, a directed acyclic graph) where data is processed from input to output. This includes reading input data, transforming that data, and writing output data. A pipeline is constructed by a user in their SDK of choice. Then, the pipeline makes its way to the runner either through the SDK directly or through the Runner API’s RPC interface. 
[Link pattern](https://beam.apache.org/documentation/patterns/overview/)

#####Pipelines are built using PCollection and PTransform in detail the firs is used for collect data batch and data stream instead the second is used as container for describe the transformation of data, it's applied to a PCollection.
PCollection is an unordered bag of elements. Multiple pipelines cannot share a PCollection. Beam pipelines process PCollections, and the runner is responsible for storing these elements.

A PCollection generally contains “big data” (too much data to fit in memory on a single machine). Therefore, the elements of a PCollection cannot be processed individually, and are instead processed uniformly in parallel.
A PCollection can be either bounded or unbounded.
A bounded PCollection is a dataset of a known, fixed size (alternatively, a dataset that is not growing over time). Bounded data can be processed by batch pipelines.
An unbounded PCollection is a dataset that grows over time, and the elements are processed as they arrive. Unbounded data must be processed by streaming pipelines.

In [ ]:
!pip install apache_beam